In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

def covariance(i, H):
    if i==0:
        return 1
    else:
        return ((i-1)**(2*H) - 2*i**(2*H) + (i+1)**(2*H))/2

def fBM_Levinson(m, H, L=1, cumm=1,_seed=None):
    k=np.array(range(m))
    if _seed:
        np.random.seed(_seed)
    scaling = (L/m) ** (2*H)
    
    # -- Covariance
    cov = [covariance(i, H) for i in range(m)]
    
    # -- Initialization of the algorithm
    y = np.random.normal(0,1,m)
    fGn = np.zeros(m)
    v1 = np.array(cov)
    v2 = np.array([0] + cov[1:] + [0])
    k = v2[1]
    aa = np.sqrt(cov[0])
    
    # -- Levinson's algorithm
    for j in range(1,m):
        aa = aa * np.sqrt(1 - k*k)
        v = k * v2[j:m] + v1[j-1:m-1]
        v2[j:m] = v2[j:m] + k * v1[j-1:m-1]
        v1[j:m] = v
        bb = y[j] / aa
        fGn[j:m] = fGn[j:m] + bb * v1[j:m]
        k = -v2[j+1]/(aa*aa)
    
    # -- scaling and output
    for i in range(m):
        fGn[i] = scaling * fGn[i]
        if cumm and i>0:
            fGn[i] = fGn[i] + fGn[i-1]
    
    return fGn


In [3]:
#Input params
m = 1000
H = 0.3
cumm = 1
L = m
seed = 5543

s = np.zeros(30)
for i in range(30):
    output = fBM_Levinson(m,H,L, _seed=None,cumm=cumm)
    s[i] = np.std(output)

array([  4.57524248,   7.17533521,   5.6030677 ,   8.94146202,
         3.63444952,   6.95915401,   7.43364316,   6.19381467,
         9.91363146,   6.29934209,   5.3788057 ,   4.01106955,
         8.88676262,  15.60599001,   8.60361862,   5.51126346,
         8.17543116,   7.37624635,  10.56650595,  11.03216768,
        16.62665377,   7.50378581,   7.63730914,  15.40024266,
         8.7179683 ,   5.0159027 ,   3.67457114,  11.15849054,
         6.08300485,   3.23098284])